In [1]:
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import cv2
from pathlib import Path
import numpy as np
import shutil

from tools import upscale_image, upscale_mask

In [2]:
src_dir = Path('../dev_gen/one_frame_episodes')
meta_config_path = src_dir.parent / "MetaConfigCheckpoint.cs"
dst_dir = Path('../gen_results/one_frame_upscaled')


image_folders = ['Gopro']
mask_specifications = [('GoproFullSegMask', 2), ('GoproObjectOnlySegMask', 2), ('HorizonGopro', 0)]
to_copy_folders = ['configs']

In [3]:
assert not dst_dir.exists()

In [4]:
ref_image = cv2.imread('reference_image.JPG', cv2.IMREAD_UNCHANGED)
print(ref_image.shape)

(3888, 5184, 3)


# upscale and compress images

In [5]:
for image_dir_name in image_folders:
    print(image_dir_name)
    for image_path in tqdm(glob(str(src_dir / image_dir_name / "**"))):
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        image = upscale_image(image)
        
        assert image.shape == ref_image.shape

        save_to_path = dst_dir / image_dir_name / Path(image_path).with_suffix('.JPG').name
        save_to_path.parent.mkdir(exist_ok=True, parents=True)
        if not cv2.imwrite(save_to_path, image, [cv2.IMWRITE_JPEG_QUALITY, 93]):
            raise ValueError("something went wrong and image wasn't saved")

Gopro


100%|██████████| 5000/5000 [22:06<00:00,  3.77it/s]


# masks

In [6]:
for mask_dir_name, channel_idx in mask_specifications:
    print(mask_dir_name)
    for mask_path in tqdm(glob(str(src_dir / mask_dir_name / "**"))):
        mask_like = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)[:, :, channel_idx]
        mask = upscale_mask(mask_like)
        
        assert mask.shape == ref_image.shape[:2]

        save_to_path = dst_dir / mask_dir_name / Path(mask_path).name
        save_to_path.parent.mkdir(exist_ok=True, parents=True)
        if not cv2.imwrite(save_to_path, mask):
            raise ValueError("something went wrong and image wasn't saved")

GoproFullSegMask


100%|██████████| 5000/5000 [09:35<00:00,  8.69it/s]


GoproObjectOnlySegMask


100%|██████████| 5000/5000 [09:36<00:00,  8.67it/s]


HorizonGopro


100%|██████████| 5000/5000 [12:52<00:00,  6.47it/s]


In [7]:
for mask_path in glob(str(dst_dir / mask_specifications[0][0] / "**")):
    resaved_mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    print(resaved_mask.shape, resaved_mask.dtype)
    print(np.unique(resaved_mask))
    break

(3888, 5184) uint8
[  0 255]


# copy other stuff

In [8]:
for to_copy_dir_name in to_copy_folders:
    print(to_copy_dir_name)
    shutil.copytree(src_dir / to_copy_dir_name, dst_dir / to_copy_dir_name)

configs


In [9]:
shutil.copyfile(meta_config_path, dst_dir / meta_config_path.name)

WindowsPath('../gen_results/one_frame_upscaled/MetaConfigCheckpoint.cs')